# Video Processing for BT Lab
Converts a video into png files. 
<br>
<br>
Usage: run notebook 
<br>
<br>
Notes: looks for a couple of JSON files to find video data.
<br>
<br>
![UofC logo](./pictures/uofc_logo-black.jpg)

In [17]:
#import libraries
import os
import cv2
import json
import pandas as pd
import numpy as np
from timeit import default_timer as timer
import psutil
import multiprocessing

In [18]:
# make sure to update path
user_drive = input("Enter user drive: ").upper()
video_path = f"{user_drive}:/Christian/DI_centre_structured"
input(f"Is this the right directory - {video_path}?")

''

In [19]:
repo_dir = os.getcwd()
json_dir = repo_dir + "/records/JSON"

In [20]:
""" creates folder for exported frames if it does not exist """
def set_folder(save_folder:str) -> None:
    if not os.path.isdir(save_folder):
        os.mkdir(save_folder)
    else:
        for files in os.listdir(save_folder):
            os.remove(os.path.join(save_folder, files))

In [21]:
""" gets drives storage """
def get_drive_storage(drive_letter):
    try:
        drive_usage = psutil.disk_usage(drive_letter + ":\\")
        
        # Convert bytes to GB
        total_space = drive_usage.total / (1024**3)
        used_space = drive_usage.used / (1024**3)
        free_space = drive_usage.free / (1024**3)
        
        return {
            'drive_letter': drive_letter,
            'total_space_GB': total_space,
            'used_space_GB': used_space,
            'free_space_GB': free_space,
        }
    except FileNotFoundError:
        return f"Drive {drive_letter} not found."

In [22]:
class NotEnoughFreeSpace(Exception):
    pass

In [23]:
""" converts video into frames """
def run_video_to_frame(video_path:str , save_folder:str, frame_frequency: dict, new_fps:int) -> None:
    # open video file
    video = cv2.VideoCapture(video_path)
    assert video.isOpened()

    # unpack metadata
    vid_fps = int(video.get(cv2.CAP_PROP_FPS))
    number_of_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
    vid_length_floored = number_of_frames // vid_fps

    # local variables
    pick_counter, save_counter, set_counter, true_frames = [0, 0, 0, 0]
    list_of_frames = [frame for frame, frequency in frame_frequency.items()]
    expected_frames = new_fps*vid_length_floored
    success = True

    # only for testing
    frame_limit = 100

    # make folder if it doesn't exist
    set_folder(save_folder)
    
    #  print to terminal
    print(f"\nSaving frames to: {save_folder}\nPicking frames: {list_of_frames} per set\nExpected number of frames: {expected_frames} ({new_fps}FPS * {vid_length_floored}s).")
    
    # start video processing
    start_time  = timer()
    while success and set_counter < vid_length_floored:
        success, frame = video.read()

        if not success:
            break

        if pick_counter in list_of_frames:
            frame_name = "frame_%d.png" % true_frames
            frame_path = os.path.join(save_folder, frame_name)
            num_of_save = frame_frequency.get(pick_counter)

            for _ in range(num_of_save):
                cv2.imwrite(frame_path, frame)
                save_counter += 1

        pick_counter += 1
        true_frames += 1

        if (pick_counter == vid_fps):
            set_counter += 1
            pick_counter = 0

        # for testing
        # if true_frames > frame_limit:
        #     print(f"\nLimit set to {frame_limit} frames for testing purposes.")
        #     break

    end_time  = timer()
    time_delta = end_time - start_time
    print(f"\nDone in {time_delta} seconds.")
    video.release()

    if save_counter != expected_frames:
        raise ValueError(f"Expected {expected_frames} frames, but got {save_counter} frames")

    return [set_counter, save_counter, true_frames]

In [24]:
""" cleans path """
def get_frames_path(local_path: str, level:str) -> str:
    fixed_path = local_path.replace("\\", "/")
    fixed_path_split = fixed_path.split("/")
    video_folder = "/".join(fixed_path_split[:-1])
    video_filename = fixed_path_split[-1].split(".")[0]
    folder_path = video_folder + f"/frames_{video_filename}_{level}"
    return folder_path

In [25]:
""" re samples number of frames """
def up_sample(old_fps: int, new_fps: int) -> list[int]:
    frames_arr = np.arange(0, old_fps, dtype=int)
    frames_interp = np.linspace(0, old_fps - 1, new_fps)
    nearest_indices = np.round(frames_interp).astype(int)
    up_sampled_list = np.take(frames_arr, nearest_indices, mode='wrap')

    return up_sampled_list.tolist()

In [26]:
def run_function_with_cpu_limit(func, arg, num_cpus):
    total_cpus = multiprocessing.cpu_count()
    num_cpus = min(num_cpus, total_cpus)
    
    processes = []
    
    for i in range(num_cpus):
        process = multiprocessing.Process(target=func, args=(arg,))
        processes.append(process)
        process.start()
    
    # Wait for all processes to finish
    for process in processes:
        process.join()

In [27]:
def run_function_with_cpu_limit(func, arg1, arg2, arg3, arg4, num_cpus):
    max_cpu = 8
    num_cpus = min(num_cpus, max_cpu)
    processes = []
    
    for i in range(num_cpus):
        process = multiprocessing.Process(target=func, args=(arg1, arg2, arg3, arg4))
        processes.append(process)
        process.start()
    
    for process in processes:
        process.join()

In [33]:
def is_png_corrupted(file_path):
    try:
        image = cv2.imread(file_path, cv2.IMREAD_UNCHANGED)
        if image is None:
            return True
        return False
    except Exception:
        return True

def check_for_corrupted_png_files(folder_path):
    corrupted_files = []
    for root, _, files in os.walk(folder_path):
        for file in files:
            if file.lower().endswith(".png"):
                file_path = os.path.join(root, file)
                if is_png_corrupted(file_path):
                    corrupted_files.append(file_path)

    return corrupted_files

folder_path = "C:/Christian/DI_centre_structured/DI_CAMERA_P3225/Final/Arun/2 Meters/With Blankets/Hold Breath/frames_Arun2_lower_bound"
corrupted_files = check_for_corrupted_png_files(folder_path)

if len(corrupted_files) == 0:
    print("No corrupted PNG files found.")
else:
    print("Corrupted PNG files found:")
    for file_path in corrupted_files:
        print(file_path)

Corrupted PNG files found:
C:/Christian/DI_centre_structured/DI_CAMERA_P3225/Final/Arun/2 Meters/With Blankets/Hold Breath/frames_Arun2_lower_bound\frame_6969.png


In [28]:
""" driver code for converting a video into frames """
def convert_video_to_frame(all_patients:dict, level: str, new_fps:int, user_drive:str) -> None:
    for json_index, patient_info in all_patients.items():
        try:
            minimum_space = 50
            video_path = patient_info["local path"]
            old_fps = int(patient_info["old fps"])

            frames_folder = get_frames_path(video_path, level)
            
            frames_to_pick = up_sample(old_fps, new_fps)
            frames_idx = pd.Index(frames_to_pick, name="frames")
            frame_frequency = frames_idx.value_counts()

            if len(frames_to_pick) != new_fps:
                raise ValueError("Number of frames to pick is not equal to new fps")

            # run video to frames
            set_counter, save_counter, true_frames = run_video_to_frame(video_path, frames_folder, frame_frequency, new_fps)

            #display output
            print(f"\nSet counter: {set_counter}, save counter: {save_counter}, frame counter: {true_frames}\n\n"+ "-"*50)

            # exit if drive storage has less than 50 GB
            drive_info = get_drive_storage(user_drive)
            if drive_info["free_space_GB"] < minimum_space:
                print("here")
                raise NotEnoughFreeSpace(f"Not enough free space on Drive {user_drive}. Minimum {minimum_space} GB required.")

        except Exception as e:
            print(f'''{type(e)}: {e} for video {patient_info["filename"]}''')

        break

In [29]:
""" reads JSON file """
def load_json(json_dir:str, filename:str) -> dict:
    full_path = json_dir + "/" + filename

    with open(full_path, "r") as json_data:
        data = json.load(json_data)

    return(data)

## Video to frames (stage 3.1)

Based on the desired fps, turn videos into frames

In [30]:
""" load JSON files """

metadata_rgb = load_json(json_dir, "rgb_test.json")
metadata_thermal = load_json(json_dir, "thermal_test.json")

In [31]:
""" convert video to png (rgb) """

for level, new_fps in rgb_fps.items():
    print(f"\nAdjusting FPS to {new_fps}\n" + "="*50)
    convert_video_to_frame(metadata_rgb, level, new_fps, user_drive)
    break


Adjusting FPS to 10



Saving frames to: C:/Christian/DI_centre_structured/DI_CAMERA_P3225/Final/Arun/2 Meters/With Blankets/Hold Breath/frames_Arun2_lower_bound
Picking frames: [0, 3, 6, 9, 12, 16, 19, 22, 25, 28] per set
Expected number of frames: 740 (10FPS * 74s).
frame_0.png is corrupted
frame_3.png is corrupted
frame_6.png is corrupted
frame_9.png is corrupted
frame_12.png is corrupted
frame_16.png is corrupted
frame_19.png is corrupted
frame_22.png is corrupted
frame_25.png is corrupted
frame_28.png is corrupted
frame_29.png is corrupted
frame_32.png is corrupted
frame_35.png is corrupted
frame_38.png is corrupted
frame_41.png is corrupted
frame_45.png is corrupted
frame_48.png is corrupted
frame_51.png is corrupted
frame_54.png is corrupted
frame_57.png is corrupted
frame_58.png is corrupted
frame_61.png is corrupted
frame_64.png is corrupted
frame_67.png is corrupted
frame_70.png is corrupted
frame_74.png is corrupted
frame_77.png is corrupted
frame_80.png is corrupted
frame_83.png is corrupted
fra